In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Data loading and cleaning

In [2]:
df = pd.read_csv('data/american_bankruptcy_dataset.csv')

In [3]:
df.head()

,company_name,fyear,status_label,X1,X2,X3,X4,X5,X6,X7,...,X11,X12,X13,X14,X15,X16,X17,X18,Division,MajorGroup
0,C_1,1999.0,alive,511267.0,740998.000,833107.0,180447.0,18373.00,70658.00,89031.000,...,35.163,201026.000,128.348,1024333.000,372.7519,401483.0,1024333.000,935302.0,D,37
1,C_1,2000.0,alive,485856.0,701.854,713811.0,179987.0,18577.00,45.79,64367.000,...,18531.000,204065.000,115187.000,874255.000,377.1180,361642.0,874255.000,809888.0,D,37
2,C_1,2001.0,alive,436656.0,710199.000,526477.0,217699.0,22496.00,4711.00,27207.000,...,-58.939,139.603,77528.000,638721.000,364.5928,399964.0,638721.000,611514.0,D,37
3,C_1,2002.0,alive,396412.0,686.621,496747.0,164658.0,27172.00,3573.00,30745.000,...,-12.410,124106.000,66322.000,606337.000,143.3295,391633.0,606337.000,575592.0,D,37
4,C_1,2003.0,alive,432204.0,709.292,523302.0,248666.0,26.68,20811.00,47.491,...,3504.000,131884.000,104661.000,651.958,308.9071,407608.0,651.958,604467.0,D,37


In [4]:
df.columns

Index(['company_name', 'fyear', 'status_label', 'X1', 'X2', 'X3', 'X4', 'X5',
       'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16',
       'X17', 'X18', 'Division', 'MajorGroup'],
      dtype='object')

Rename columns

In [5]:
# Dictionary mapping original column names to descriptive names
column_rename_dict = {
    'X1': 'current_assets',
    'X2': 'cost_of_goods_sold',
    'X3': 'depreciation_amortization',
    'X4': 'ebitda',
    'X5': 'inventory',
    'X6': 'net_income',
    'X7': 'total_receivables',
    'X8': 'market_value',
    'X9': 'net_sales',
    'X10': 'total_assets',
    'X11': 'total_long_term_debt',
    'X12': 'ebit',
    'X13': 'gross_profit',
    'X14': 'total_current_liabilities',
    'X15': 'retained_earnings',
    'X16': 'total_revenue',
    'X17': 'total_liabilities',
    'X18': 'total_operating_expenses',
    'MajorGroup': 'major_group'
}

# Rename the columns
df = df.rename(columns=column_rename_dict)
df.columns = df.columns.str.lower()

df['company_name'] = pd.to_numeric(df['company_name'].str.replace('C_', ''))

df['status_label'] = df['status_label'] == 'failed'

df['fyear'] = df['fyear'].astype(int)

In [6]:
df.head()

,company_name,fyear,status_label,current_assets,cost_of_goods_sold,depreciation_amortization,ebitda,inventory,net_income,total_receivables,...,total_long_term_debt,ebit,gross_profit,total_current_liabilities,retained_earnings,total_revenue,total_liabilities,total_operating_expenses,division,major_group
0,1,1999,False,511267.0,740998.000,833107.0,180447.0,18373.00,70658.00,89031.000,...,35.163,201026.000,128.348,1024333.000,372.7519,401483.0,1024333.000,935302.0,D,37
1,1,2000,False,485856.0,701.854,713811.0,179987.0,18577.00,45.79,64367.000,...,18531.000,204065.000,115187.000,874255.000,377.1180,361642.0,874255.000,809888.0,D,37
2,1,2001,False,436656.0,710199.000,526477.0,217699.0,22496.00,4711.00,27207.000,...,-58.939,139.603,77528.000,638721.000,364.5928,399964.0,638721.000,611514.0,D,37
3,1,2002,False,396412.0,686.621,496747.0,164658.0,27172.00,3573.00,30745.000,...,-12.410,124106.000,66322.000,606337.000,143.3295,391633.0,606337.000,575592.0,D,37
4,1,2003,False,432204.0,709.292,523302.0,248666.0,26.68,20811.00,47.491,...,3504.000,131884.000,104661.000,651.958,308.9071,407608.0,651.958,604467.0,D,37


# EDA

In [7]:
df.shape

(78682, 23)

In [8]:
df['company_name'].nunique()

8971

In [9]:
df['status_label'].value_counts(normalize=True)

status_label
False    0.933657
True     0.066343
Name: proportion, dtype: float64

In [10]:
df['division'].value_counts(normalize=True)

division
D    0.487837
I    0.217941
E    0.091546
G    0.072380
B    0.053138
F    0.037353
H    0.026181
C    0.009163
A    0.004296
J    0.000165
Name: proportion, dtype: float64

The division applies to the company as a whole, so presumably it's the sector they're in.

In [11]:
df.groupby('company_name')['division'].nunique().max()

np.int64(1)

In [12]:
division_le = LabelEncoder()
df['division'] = division_le.fit_transform(df['division'])

In [13]:
df['major_group'].value_counts(normalize=True)

major_group
73    0.142167
28    0.107496
36    0.090402
38    0.081721
35    0.057955
        ...   
41    0.000254
8     0.000254
99    0.000165
46    0.000089
86    0.000051
Name: proportion, Length: 73, dtype: float64

In [14]:
df.groupby('company_name')['major_group'].nunique().max()

np.int64(1)

Similar comment here

In [15]:
major_group_le = LabelEncoder()
df['major_group'] = major_group_le.fit_transform(df['major_group'])

Right now I just label encoded, but if either of these columns end up being useful we should instead one-hot encode them.

## Missing values

In [16]:
df.isna().sum().max()

np.int64(0)

In [17]:
df.columns[df.max() == np.inf]

Index([], dtype='object')

# Feature engineering

In [18]:
df['R_GrossProfitMargin'] = df['gross_profit']/df['total_revenue']
df['R_NetProfitMargin'] = df['net_income']/df['total_revenue']
df['R_ReturnOnAssets'] = df['net_income']/df['total_assets']
df['R_ReturnOnEquity'] = df['net_income']/(df['total_assets'] - df['total_liabilities'])
df['R_CurrentRatio'] = df['current_assets']/df['total_current_liabilities']
df['R_QuickRatio'] = (df['current_assets'] - df['inventory'])/df['total_current_liabilities']
df['R_DebtToEquityRatio'] = df['total_liabilities']/(df['total_assets'] - df['total_liabilities'])
df['R_DebtRatio'] = df['total_liabilities']/df['total_assets']
df['R_InterestCoverageRatio'] = df['ebit']/df['total_long_term_debt']
df['R_CashFlowToDebtRatio'] = df['ebitda']/df['total_liabilities']

In [19]:
identifier_cols = ['company_name', 'fyear', 'status_label', 'division', 'major_group']
ratio_cols = [col for col in df.columns if col.startswith('R_')]

ratio_df = df[identifier_cols + ratio_cols]

In [20]:
ratio_df.columns[ratio_df.max() == np.inf]

Index(['R_DebtToEquityRatio', 'R_InterestCoverageRatio'], dtype='object')

These two columns have `inf` values, probably because we're dividing by zero. Two companies have assets equaling liabilities.

In [21]:
(df['total_assets'] == df['total_liabilities']).sum()

np.int64(2)

In [22]:
df[df['total_assets'] == df['total_liabilities']][['total_assets', 'total_liabilities']]

,total_assets,total_liabilities
56207,629.0,629.0
76050,584.0,584.0


Replace these debt to asset ratios with the value at the 99th percentile.

In [23]:
ratio_df.loc[ratio_df['R_DebtToEquityRatio'] > ratio_df['R_DebtToEquityRatio'].quantile(0.99), 'R_DebtToEquityRatio'] = ratio_df['R_DebtToEquityRatio'].quantile(0.99)
ratio_df.loc[ratio_df['R_InterestCoverageRatio'] > ratio_df['R_InterestCoverageRatio'].quantile(0.99), 'R_InterestCoverageRatio'] = ratio_df['R_InterestCoverageRatio'].quantile(0.99)

In [24]:
ratio_df.loc[ratio_df['R_DebtToEquityRatio'] < ratio_df['R_DebtToEquityRatio'].quantile(0.01), 'R_DebtToEquityRatio'] = ratio_df['R_DebtToEquityRatio'].quantile(0.01)
ratio_df.loc[ratio_df['R_InterestCoverageRatio'] < ratio_df['R_InterestCoverageRatio'].quantile(0.01), 'R_InterestCoverageRatio'] = ratio_df['R_InterestCoverageRatio'].quantile(0.01)
ratio_df.loc[ratio_df['R_ReturnOnEquity'] < ratio_df['R_ReturnOnEquity'].quantile(0.01), 'R_ReturnOnEquity'] = ratio_df['R_ReturnOnEquity'].quantile(0.01)

In [25]:
ratio_df.head()

,company_name,fyear,status_label,division,major_group,R_GrossProfitMargin,R_NetProfitMargin,R_ReturnOnAssets,R_ReturnOnEquity,R_CurrentRatio,R_QuickRatio,R_DebtToEquityRatio,R_DebtRatio,R_InterestCoverageRatio,R_CashFlowToDebtRatio
0,1,1999,False,3,29,0.000320,0.175993,0.431325,-0.082111,0.499122,0.481185,-1.190369,6.252948,5716.975230,0.176160
1,1,2000,False,3,29,0.318511,0.000127,0.000365,-0.000061,0.555737,0.534488,-1.167443,6.972175,11.012088,0.205875
2,1,2001,False,3,29,0.193837,0.011779,0.031310,-0.009649,0.683641,0.648421,-1.308166,4.245009,-2.368601,0.340836
3,1,2002,False,3,29,0.169347,0.009123,0.017551,-0.008871,0.653782,0.608968,-1.505447,2.978445,-10000.483481,0.271562
4,1,2003,False,3,29,0.256769,0.051056,0.158547,0.159338,662.932275,662.891352,0.004992,0.004967,37.638128,381.414140


In [26]:
assert ratio_df.isna().sum().max() == 0, "Missing values in ratio_df"
assert (ratio_df.max() == np.inf).sum().max() == 0, "inf values in ratio_df"
assert (ratio_df.min() == -np.inf).sum().max() == 0, "-inf values in ratio_df"

Scaling

In [27]:
scaler = StandardScaler()
ratio_df.loc[:, ratio_cols] = scaler.fit_transform(ratio_df.loc[:, ratio_cols])

Quantization

In [28]:
ratio_df.loc[:, ratio_df.select_dtypes('float').columns] = ratio_df.select_dtypes('float').astype('float16')
ratio_df.loc[:, ratio_df.select_dtypes('int').columns] = ratio_df.select_dtypes('int').astype('uint16')

In [29]:
ratio_df.to_csv('data/american_bankruptcy_dataset_ratio.csv', index=False)